In [12]:
import pandas as pd
import rasterio
import rasterio.mask
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import fiona
import contextily as cx
from potentiel_solaire.constants import DATA_FOLDER
from scipy.ndimage import sobel, gaussian_filter, shift, label, morphology
import os

In [6]:
# Verification des layers disponibles pour saint-denis
stdenis_path = DATA_FOLDER / "saint_denis_reference_data.gpkg"
layers = fiona.listlayers(stdenis_path)
print(" ".join(layers))

cadastre_parcellaire potentielsolaire_bati potentielsolaire_toitures perimetre_st_denis identifiers annuaire_education annuaire_education_sans_zone bdtopo_education bdtopo_batiment


In [ ]:
# On va utiliser le script pour chaque zone, concaténer les résultats de slopes et regarder si les écoles sont toutes plates ou dans la majorité
# On va définir un seuil de surface pour considérer qu'une école est plate
# On va aussi, faire une distribution des pentes des rasters sur les écoles
# On va regarder le pourcentage de surface utile

In [32]:
# On ouvre une zone
stdenis = gpd.read_file(stdenis_path, layer="bdtopo_education")# toponymie_services_et_activites
stdenis = stdenis.to_crs(2154) # 2154 Lambert, 4326 latlon  
l = len(stdenis)
slopes = {}
slopes_means = {}
percentage_under_10 = {}

for i in range(len(stdenis)):
    ID = i
    example = stdenis[ID:ID+1]
    ID = example["cleabs_left"].iloc[0]
    NOM = example["toponyme"].iloc[0]
    
    # On ouvre les toitures
    toiture = gpd.read_file(stdenis_path, layer="bdtopo_batiment")# toponymie_services_et_activites
    toiture_b = gpd.read_file(stdenis_path, layer="potentielsolaire_bati")# toponymie_services_et_activites
    toiture_t = gpd.read_file(stdenis_path, layer="potentielsolaire_toitures")# toponymie_services_et_activites
    toiture = toiture.to_crs(2154) # 2154 Lambert, 4326 latlon  
    toiture_b = toiture_b.to_crs(2154)
    toiture_t = toiture_t.to_crs(2154)
    toiture = gpd.sjoin(toiture, example, how='inner', predicate='intersects',lsuffix='_li', rsuffix='_ri').clip(example)
    toiture_b = gpd.sjoin(toiture_b, example, how='inner', predicate='intersects',lsuffix='_li', rsuffix='_ri').clip(example)
    toiture_t = gpd.sjoin(toiture_t, example, how='inner', predicate='intersects',lsuffix='_li', rsuffix='_ri').clip(example)
    geotiff_cached_toits = "../data/cache/mns/"+ID+"_toits.masked.tif"

    if not os.path.isfile(geotiff_cached_toits) or True:
        geome = example.geometry.total_bounds
        A = "0"+str(int(geome[0]//1000))
        B = str(int(geome[1]//1000)+1)
        filename = "MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-"+A+"-"+B+"-LA93-0M50"
        tile = "/Users/sarahlenet/Desktop/potentiel_solaire/13_potentiel_solaire/algorithme/data/MNS-Correl_1-0__TIFF_LAMB93_D093_2024-01-01/MNS-Correl/1_DONNEES_LIVRAISON_2024-11-00179/MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-"+A+"-"+B+"-LA93-0M50.tif"
        path = DATA_FOLDER / tile
        with rasterio.open(str(path)) as img:
            try:
                out_image, out_transform = rasterio.mask.mask(img, toiture.geometry, crop=True)
                out_meta = img.meta
            except:
                slopes[filename] = None
        minval = np.min(out_image[np.nonzero(out_image)])
        out_image = out_image - minval
    
        out_image = np.where(out_image<0, 0, out_image)
    
        MAX = np.percentile(out_image[np.nonzero(out_image)],80)
        out_image = np.where(out_image>=MAX, MAX, out_image)
    
        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})

        with rasterio.open("/Users/sarahlenet/Desktop/potentiel_solaire/13_potentiel_solaire/algorithme/data/cache/mns/"+ID+".masked.tif", "w", **out_meta) as dest:
           dest.write(out_image)
        
        # Calcul des gradients X et Y
        dy, dx = np.gradient(out_image[0])
        
        dx = sobel(out_image[0], axis=1)  # Gradient selon l'axe X (est-ouest)
        dy = sobel(out_image[0], axis=0)  # Gradient selon l'axe Y (nord-sud)
        slope = np.sqrt(dx**2 + dy**2)
        threshold = 2  # Seuil en mètres (à ajuster selon la donnée)
        mask = out_image[0] < threshold

        slope_cleaned = np.where(mask, np.nan, np.arctan(np.sqrt(dx**2 + dy**2)) * (180 / np.pi)) # np.nan sur ce qui n'est pas bâtiment
        slopes[filename] = slope_cleaned # dictionnaire des slopes
        slopes_means[filename] = np.nanmean(slope_cleaned) # dictionnaire des moyennes des slopes sur les non-nan values
        valid_values = slope_cleaned[~np.isnan(slope_cleaned)] 
        count_under_10 = np.sum(valid_values < 10)
        percentage_under_10_value = (count_under_10 / len(valid_values)) * 100
        percentage_under_10[filename] = percentage_under_10_value # dictionnaire des valeurs en-dessous de 10 degrés (arbitraire, on pourrait chercher les pentes pour lesquelles on considère que c'est plat)

/var/folders/vm/ylnr3pdd3pjdxlnt2mn3hy7c0000gn/T/ipykernel_10926/2931344784.py:69: RuntimeWarning: Mean of empty slice
  slopes_means[filename] = np.nanmean(slope_cleaned)
/var/folders/vm/ylnr3pdd3pjdxlnt2mn3hy7c0000gn/T/ipykernel_10926/2931344784.py:72: RuntimeWarning: invalid value encountered in scalar divide
  percentage_under_10_value = (count_under_10 / len(valid_values)) * 100
/var/folders/vm/ylnr3pdd3pjdxlnt2mn3hy7c0000gn/T/ipykernel_10926/2931344784.py:69: RuntimeWarning: Mean of empty slice
  slopes_means[filename] = np.nanmean(slope_cleaned)
/var/folders/vm/ylnr3pdd3pjdxlnt2mn3hy7c0000gn/T/ipykernel_10926/2931344784.py:72: RuntimeWarning: invalid value encountered in scalar divide
  percentage_under_10_value = (count_under_10 / len(valid_values)) * 100
/var/folders/vm/ylnr3pdd3pjdxlnt2mn3hy7c0000gn/T/ipykernel_10926/2931344784.py:69: RuntimeWarning: Mean of empty slice
  slopes_means[filename] = np.nanmean(slope_cleaned)
/var/folders/vm/ylnr3pdd3pjdxlnt2mn3hy7c0000gn/T/ipyke

In [34]:
percentage_under_10

{'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0653-6870-LA93-0M50': np.float64(17.539863325740317),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0654-6870-LA93-0M50': np.float64(16.27906976744186),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0653-6871-LA93-0M50': np.float64(27.996289424860855),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0651-6871-LA93-0M50': np.float64(23.5415752866721),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0651-6870-LA93-0M50': np.float64(22.784478238070268),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0652-6872-LA93-0M50': np.float64(17.654557042702358),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0652-6873-LA93-0M50': np.float64(29.31770890373453),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0652-6869-LA93-0M50': np.float64(15.617468427648703),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0653-6869-LA93-0M50': np.float64(23.121287021378084),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0652-6871-LA93-0M50': np.float64(19.335693461057893),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0652-6868-LA9

In [ ]:
# Tuner percentage under 10 pour avoir seulement des surfaces et non la somme des pixels

In [35]:
slopes_means

{'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0653-6870-LA93-0M50': np.float32(47.601437),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0654-6870-LA93-0M50': np.float32(33.55983),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0653-6871-LA93-0M50': np.float32(31.109798),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0651-6871-LA93-0M50': np.float32(37.02075),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0651-6870-LA93-0M50': np.float32(35.588734),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0652-6872-LA93-0M50': np.float32(51.544815),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0652-6873-LA93-0M50': np.float32(32.026024),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0652-6869-LA93-0M50': np.float32(42.169025),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0653-6869-LA93-0M50': np.float32(36.069736),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0652-6871-LA93-0M50': np.float32(45.982075),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0652-6868-LA93-0M50': np.float32(31.053703),
 'MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-0653-6872-LA93-